In [1]:
import pandas as pd
import os
import numpy as np

## combined.xlsx analysis

In [45]:
base_folder_path = os.getcwd().split(os.sep + 'src')[0]
dataset_path = os.path.join(base_folder_path, 'dataset')
print("Dataset Path: " + dataset_path)

xlsx_file_path = dataset_path + os.sep + "combined.xlsx"
result_file_path = dataset_path + os.sep + "combined_analysis.xlsx"

Dataset Path: /Users/sajed/PycharmProjects/Study-on-ChatGPT/dataset


In [46]:
def find_response_type_dict(df:pd.DataFrame, counter):
    tag_dict = dict()
    explanations = ["EC", "EIC", "EPC"]
    answers = ['AC', 'AIC', 'APC']
    for tag_1 in explanations:
        for tag_2 in answers:
            tag_dict[ tag_1 + '-' + tag_2] = 0

    for index, row in df.iterrows():
        if row['Explanation.'+str(counter)] not in explanations:
            continue
        if row['Answer.'+str(counter)] not in answers:
            continue

        key = row['Explanation.'+str(counter)] + '-' + row['Answer.'+str(counter)]

        if key in tag_dict:
            tag_dict[key] += 1

    return tag_dict

In [47]:
sheets = pd.ExcelFile(os.path.join(xlsx_file_path)).sheet_names
total_iterations = 3
for sheet in sheets:
    df = pd.read_excel(xlsx_file_path, sheet)
    print( "Sheet:", sheet)

    result = pd.DataFrame()
    for index in range(1, (total_iterations*2)+1):
        tag_dict = find_response_type_dict(df, index)
        result = result.append( pd.DataFrame( [tag_dict]) )

    index_column = pd.Index(['shared_1', 'shared_2', 'shared_3', 'separate_1', 'separate_2', 'separate_3'])
    result = result.set_index(index_column)
    print(result)
    # with pd.ExcelWriter(result_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    #     result.to_excel(writer, sheet_name=sheet+"_testing", index=True)

Sheet: all
            EC-AC  EC-AIC  EC-APC  EIC-AC  EIC-AIC  EIC-APC  EPC-AC  EPC-AIC  \
shared_1       13       0       0       2       11        0       0        2   
shared_2       14       0       1       1       13        0       0        1   
shared_3       13       0       1       1       12        0       1        1   
separate_1     18       0       0       0       14        0       0        5   
separate_2     19       0       1       0       14        0       1        3   
separate_3     21       0       2       0       13        0       0        2   

            EPC-APC  
shared_1          3  
shared_2          1  
shared_3          2  
separate_1        3  
separate_2        2  
separate_3        2  


/var/folders/0j/xlmdg4yd2m3_d3s69ffh30ph0000gn/T/ipykernel_59478/3713119519.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append( pd.DataFrame( [tag_dict]) )
/var/folders/0j/xlmdg4yd2m3_d3s69ffh30ph0000gn/T/ipykernel_59478/3713119519.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append( pd.DataFrame( [tag_dict]) )
/var/folders/0j/xlmdg4yd2m3_d3s69ffh30ph0000gn/T/ipykernel_59478/3713119519.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append( pd.DataFrame( [tag_dict]) )
/var/folders/0j/xlmdg4yd2m3_d3s69ffh30ph0000gn/T/ipykernel_59478/3713119519.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

## Confidence analysis

In [58]:
def find_confidence_dict(df:pd.DataFrame):
    tag_dict = dict()
    tags = ["EC", "EIC", "EPC"]
    # tags = ['AC', 'AIC', 'APC']
    for tag in tags:
        tag_dict[ tag ] = dict()

    for index, row in df.iterrows():

        if pd.isna(row['Confidence.2']):
            key = row['Explanation.1']
            if key in tag_dict:
                if row['Confidence.1'] not in tag_dict[key]:
                    tag_dict[key][row['Confidence.1']] = 1
                else:
                    tag_dict[key][row['Confidence.1']] += 1
        else:
            key = row['Explanation.4']
            if key in tag_dict:
                if row['Confidence.2'] not in tag_dict[key]:
                    tag_dict[key][row['Confidence.2']] = 1
                else:
                    tag_dict[key][row['Confidence.2']] += 1

    return tag_dict

In [59]:
sheets = pd.ExcelFile(os.path.join(xlsx_file_path)).sheet_names

for sheet in sheets:
    df = pd.read_excel(xlsx_file_path, sheet)
    print( "Sheet:", sheet)

    tag_dict = find_confidence_dict(df)
    result = pd.DataFrame( tag_dict)
    result = result. replace(np.nan,0)

    print(result)

    # with pd.ExcelWriter(result_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    #     result.to_excel(writer, sheet_name=sheet+"_confidence", index=True)


Sheet: all
                  EC   EIC  EPC
Highly confident   8  10.0  2.0
Very confident     3   1.0  2.0
Confident          8   4.0  1.0
Reliable           1   0.0  0.0


## Overall Calculations

In [43]:
def find_tag_dict(df:pd.DataFrame, counter):

    # tags = ["EC", "EIC", "EPC"]
    tags = ["AC", "AIC", "APC"]
    tag_dict = dict()
    for tag in tags:
        tag_dict[tag] = 0

    for index, row in df.iterrows():
        if row['Answer.'+str(counter)] not in tags:
            continue

        tag_dict[row['Answer.'+str(counter)]] += 1

    return tag_dict

In [44]:
xlsx_file_path = dataset_path + os.sep + "combined_pair.xlsx"
sheets = pd.ExcelFile(os.path.join(xlsx_file_path)).sheet_names
total_iterations = 3

for sheet in sheets:
    df = pd.read_excel(xlsx_file_path, sheet)
    print( "Sheet:", sheet)

    result = pd.DataFrame()
    for index in range(1, (total_iterations*2)+1):
        tag_dict = find_tag_dict(df, index)
        result = result.append( pd.DataFrame( [tag_dict]) )

    index_column = pd.Index(['shared_1', 'shared_2', 'shared_3', 'separate_1', 'separate_2', 'separate_3'])
    result = result.set_index(index_column)
    print(result)
    # with pd.ExcelWriter(result_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    #     result.to_excel(writer, sheet_name=sheet+"_testing", index=True)

Sheet: all
            AC  AIC  APC
shared_1    15   13    3
shared_2    15   14    2
shared_3    15   13    3
separate_1  11   17    3
separate_2  14   14    3
separate_3  15   12    4


/var/folders/0j/xlmdg4yd2m3_d3s69ffh30ph0000gn/T/ipykernel_59478/2045189995.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append( pd.DataFrame( [tag_dict]) )
/var/folders/0j/xlmdg4yd2m3_d3s69ffh30ph0000gn/T/ipykernel_59478/2045189995.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append( pd.DataFrame( [tag_dict]) )
/var/folders/0j/xlmdg4yd2m3_d3s69ffh30ph0000gn/T/ipykernel_59478/2045189995.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append( pd.DataFrame( [tag_dict]) )
/var/folders/0j/xlmdg4yd2m3_d3s69ffh30ph0000gn/T/ipykernel_59478/2045189995.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

In [48]:
xlsx_file_path = dataset_path + os.sep + "combined.xlsx"
df = pd.read_excel(xlsx_file_path, 'all')

In [51]:
consistent = 0
inconsistent = 0

for index, row in df.iterrows():
    prev = row['Explanation.4']

    if pd.isna(prev):
        continue

    flag = False

    for iter in range(5, 7):
        col_name = row['Explanation.'+str(iter)]
        if prev is not col_name:
            inconsistent += 1
            flag = True
            break

    if flag == False:
        consistent += 1

print("Explanation", "Inconsistent:", inconsistent, "Consistent:", consistent)

Explanation Inconsistent: 13 Consistent: 27


In [53]:
consistent = 0
inconsistent = 0

for index, row in df.iterrows():
    prev = row['Answer.1']
    if pd.isna(prev):
        continue

    flag = False

    for iter in range(2, 4):
        col_name = row['Answer.'+str(iter)]
        if prev is not col_name:
            inconsistent += 1
            flag = True
            break

    if flag == False:
        consistent += 1

print("Answer", "Inconsistent:", inconsistent, "Consistent:", consistent)

Answer Inconsistent: 5 Consistent: 26
